# Using Python to Communicate with CLI tools

Written and Prepared by MJ Hardin for ChowNow Tech Talks 

Sep. 22, 2021

## `subprocess` gets the job done most times.

You can use the `subprocess` library to simply launch other applications in the OS.

In [ ]:
import subprocess

from utils import print_lines

subprocess.run(['open', '-a', 'Visual Studio Code', 'README.md'])

You can use the `subprocess` library to capture output from subprocesses and use it in your program.

In [ ]:
proc = subprocess.run(['ls', '-l'], capture_output=True)

# print_lines(proc.stdout.decode().splitlines())
for l in proc.stdout.decode().splitlines():
    if 'requirements.pkg' in l: 
        print('HAS REQUIREMENTS.TXT')

You can even use the `subprocess` library to send input to stdin!

        # greet.py
        def main():
            name = input('What is your name?\n')
            print(f'Hi, {name}\n')


        if __name__ == '__main__':
            main()


In [ ]:
proc = subprocess.Popen(['python', 'greet.py'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

out, err = proc.communicate(input='MJ'.encode())

print_lines(out.decode().splitlines())

We reach the limits of the subprocess library cabilities when we need to communicate with a subprocess more than once like when using the `units` cli tool.

In [ ]:
proc = subprocess.Popen(['units'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

out, err = proc.communicate(input='5 feet'.encode())

print_lines(out.decode().splitlines())

We expected to be able to send input, receive output, and send input again to get 5 feet converted to yards.

        $ units
        586 units, 56 prefixes
        You have: 5 feet
        You want: yards
            * 1.6666667
            / 0.6

## `pexpect` to the rescue!

Pexpect makes it possible to write Python programs that launch subprocesses and communicate bidirectionally with those subprocesses.

Let's convert units again!

Using `pexpect` we are able to specify the input prompts that we expect from a spawned subprocess. Then, when those lines are encountered, we send the correct value to the program.

In [ ]:
import pexpect

proc = pexpect.spawn('units')

# once a line that contains "You have:" is encountered stop
proc.expect('You have:')
# then send the units you'd like converted
proc.sendline('5 feet')
# then expect a line that contains "You want:"
proc.expect('You want:')
# once that line is encountered send the string "yards"
proc.sendline('yards')
# then expect "You have:" again so that we no to stop the program
proc.expect('You have:')

proc.close()    

print_lines(proc.before.decode().splitlines())

## Let's get crazy!

Let's look at a real-world ChowNow example: Two-factor authentication!

Mulholland uses `pexpect` during a two-factor authentication process. Why? Because a CLI tool being used requires the user to enter the verification code _at the command line_!

In [ ]:
import sys

import pexpect

proc = pexpect.spawn('python log_replay.py 2fa-basic.json', encoding='utf-8')
proc.logfile = sys.stdout

index = proc.expect([
            'Please enter the 6 digit code you received at.*:',
            'Please enter the 6 digit code:',
            pexpect.EOF])

if index in (0, 1):
    verfication_code = '123456'
    proc.sendline(verfication_code)
else:
    print('Text not found')
    
proc.expect(pexpect.EOF)

Expanding on this, we can even use `pexpect` to handle incorrect codes gracefully!

In [ ]:
import logging
import sys

import pexpect

from utils import try_2fa_codes

proc = pexpect.spawn('python log_replay.py 2fa-incorrect.json', encoding='utf-8')
proc.logfile = sys.stdout


index = proc.expect([
            'Please enter the 6 digit code you received at.*:',
            'Please enter the 6 digit code:',
            pexpect.EOF])

if index in (0, 1):
    count = 1
    if index in [0, 1]:
        try_2fa_codes(proc, count=count)
    elif count >= max_2fa_retries:
        raise Exception(f'No valid verification code found for')
else:
    print('Text not found')

# Recap & Additional Examples

* Occaisionally, Python programs will need to launch subprocesses.
* In some instances the program will need to interact with the subprocess indefinitely. 
* Python's stdlib `subprocess` package doesn't work for this. 
* `pexpect` is a python package that allows users to write Python programs that can interact with subprocesses.
* Additional examples can be found here https://pexpect.readthedocs.io/en/stable/examples.html

# Thanks for attending!